## Robot Pose and Beacon Positions

The robot pose $X $ is in $SE(2) $, and the beacon positions $b_k $ are in $\mathbb{R}^2 $.

The robot pose $X $ can be represented as:
$
X = \begin{bmatrix} 
\cos \theta & -\sin \theta & x \\ 
\sin \theta & \cos \theta & y \\ 
0 & 0 & 1 
\end{bmatrix}
$
where $\theta $ represents the orientation, and $(x, y) $ represents the position in $\mathbb{R}^2 $.

Each landmark $b_k $ is defined by:
$
b_k = (b_{x_k}, b_{y_k})
$
where $b_{x_k} $ and $b_{y_k} $ are the landmark coordinates in the world frame.

## Control Signal

The control signal $u $ is a twist in $se(2) $, comprising longitudinal velocity $v $ and angular velocity $\omega $, with no lateral velocity component. It is integrated over the sampling time $\Delta t $ as follows:
$
u = (v \cdot \Delta t, 0, \omega \cdot \Delta t)
$

The control $u $ is subject to additive Gaussian noise $u_{\text{noise}} $, with covariance matrix $Q $:
$
Q = \text{diag}(\sigma_v^2, \sigma_s^2, \sigma_\omega^2)
$
This noise model allows for lateral slippage $u_s $ through a non-zero $\sigma_s $.

When a control $u $ is applied, the robot pose is updated as:
$
X \leftarrow X \cdot \exp(u) = X + u
$

## Landmark Measurements

Landmark measurements are of range and bearing type, which are represented in Cartesian coordinates for simplicity. The measurement noise $n $ is Gaussian with zero mean and covariance matrix $R $.

The rigid motion transformation of a landmark measurement $y = h(X, b) = X^{-1} b $ is given by:
$
y_k = (b_{rx_k}, b_{ry_k})
$
where $(b_{rx_k}, b_{ry_k}) $ are the landmark coordinates in the robot frame.

## Parameter Definitions

Let us define the parameters as follows:

- $X $: Robot pose, $SE(2) $
- $u $: Robot control, $(v \cdot \Delta t, 0, \omega \cdot \Delta t) $ in $se(2) $
- $Q $: Covariance of control noise
- $b_k $: $k $-th landmark position in $\mathbb{R}^2 $
- $y $: Cartesian landmark measurement in the robot frame, $\mathbb{R}^2 $
- $R $: Covariance of the measurement noise

## Motion and Measurement Models

The motion and measurement models are defined as follows:

1. **Motion Model:**
   $
   X_{t+1} = f(X_t, u) = X_t \cdot \exp(w)
   $

2. **Measurement Model:**
   $
   y_k = h(X, b_k) = X^{-1} b_k
   $

## Algorithm

The following algorithm simulates the measurements and uses them to estimate the state with a Lie-based error-state Kalman filter.

The main code comprises only a `main()` function, which includes:

1. Simulation of the true state and measurements.
2. Estimation of the state using the Kalman filter.
3. Comparison of the simulated state, estimated state, and unfiltered state for evaluating estimation quality.




### Code Explanation

1. **Includes and Definitions**:
   - `manif/SE2.h`: This includes functions and structures from the `manif` library for SE(2) Lie group operations.
   - The `Array2d` and `Array3d` types define vectors in 2D and 3D, which are used for control and noise.

2. **Initialization**:
   - **Robot Pose and Covariance**:
     - `manif::SE2d X, X_simulation, X_unfiltered;`: These represent the robot's pose (position and orientation) in the `SE(2)` group.
     - `Matrix3d P;`: This is the covariance matrix of the robot pose, initialized to zero.
   - **Control and Noise**:
     - `u_nom`: The nominal control, representing `(v * dt, 0, w * dt)`.
     - `u_sigmas`: Control noise standard deviations in each direction `(v, s, w)`.
     - `U`: The control noise covariance matrix.
   - **Landmarks**:
     - Three 2D landmarks are defined in the world frame with coordinates `b0`, `b1`, and `b2`.

3. **Temporal Loop**:
   - The loop runs for 10 time steps, simulating motion, measurement, estimation, and comparison between the simulated, estimated, and unfiltered states.

#### I. Simulation Step
   - **Control Noise**:
     - `u_noise` is a random Gaussian noise vector with the specified standard deviations in `u_sigmas`.
     - `u_noisy` is the nominal control plus this noise.
   - **Pose Update**:
     - `X_simulation = X_simulation + u_simu;`: Updates the simulated robot pose. `+` here represents `SE(2)` addition using `X.rplus(u) = X * exp(u)`.
   - **Landmark Measurement**:
     - For each landmark, the code calculates the expected measurement (`y = X_simulation.inverse().act(b);`) in the robot frame.
     - Noise is added to each measurement `y`, simulating sensor noise.

#### II. Estimation Step
   - **Control Update**:
     - `X = X.plus(u_est, J_x, J_u);`: Updates the estimated pose using the noisy control `u_est`, with Jacobians `J_x` and `J_u` for the state and control.
     - **Covariance Update**:
       - `P = J_x * P * J_x.transpose() + J_u * U * J_u.transpose();`: Updates the pose covariance matrix with the propagated uncertainty from the control noise.
   - **Measurement Update**:
     - For each measured landmark:
       - `e = X.inverse(J_xi_x).act(b, J_e_xi);`: Calculates the expected position of the landmark in the robot frame.
       - `z = y - e;`: Computes the innovation (difference) between the observed and expected measurement.
       - **Kalman Gain**:
         - `K = P * H.transpose() * Z.inverse();`: Calculates the Kalman gain, which determines how much to adjust the estimate based on the measurement.
       - **Correction**:
         - `dx = K * z;`: Calculates the correction in the tangent space.
         - `X = X + dx;`: Updates the estimated pose by adding the correction.
         - `P = P - K * Z * K.transpose();`: Updates the covariance matrix by incorporating measurement information.

#### III. Unfiltered Update
   - The code also keeps an "unfiltered" version of the state (`X_unfiltered`) updated only by the control signal, without applying any corrections from measurements.

#### IV. Output Results
   - After each step, the code prints the `X_simulation`, `X`, and `X_unfiltered` states, allowing for comparison between the simulated, estimated, and unfiltered states.

